In [0]:
from pyspark.sql.functions import broadcast, col, rand
import time

In [0]:
# 브로드캐스트 조인 힌트 임계값 확인 (기본값: 10MB)
broadcast_threshold = spark.conf.get("spark.sql.autoBroadcastJoinThreshold")
print(f"\n현재 브로드캐스트 조인 임계값: {broadcast_threshold}")


현재 브로드캐스트 조인 임계값: 10485760b


In [0]:
# 작은 테이블 생성 (부서 테이블) - 브로드캐스트 대상
departments = [(i, f"Department-{i}", f"Location-{i%5}") for i in range(1, 101)]
dept_df = spark.createDataFrame(departments, ["dept_id", "dept_name", "location"])
    
dept_df.show()

+-------+-------------+----------+
|dept_id|    dept_name|  location|
+-------+-------------+----------+
|      1| Department-1|Location-1|
|      2| Department-2|Location-2|
|      3| Department-3|Location-3|
|      4| Department-4|Location-4|
|      5| Department-5|Location-0|
|      6| Department-6|Location-1|
|      7| Department-7|Location-2|
|      8| Department-8|Location-3|
|      9| Department-9|Location-4|
|     10|Department-10|Location-0|
|     11|Department-11|Location-1|
|     12|Department-12|Location-2|
|     13|Department-13|Location-3|
|     14|Department-14|Location-4|
|     15|Department-15|Location-0|
|     16|Department-16|Location-1|
|     17|Department-17|Location-2|
|     18|Department-18|Location-3|
|     19|Department-19|Location-4|
|     20|Department-20|Location-0|
+-------+-------------+----------+
only showing top 20 rows



In [0]:
# 큰 테이블 생성 (직원 테이블)
# 1천만 건의 직원 데이터 생성
num_employees = 10_000_000
employees = [(i, 
    f"Employee-{i}", 
    1 + (i % 100),  # dept_id (1-100 사이)
    50000 + (i % 50000))  # salary
    for i in range(1, num_employees + 1)]
emp_df = spark.createDataFrame(employees, ["emp_id", "name", "dept_id", "salary"])

emp_df.show()

+------+-----------+-------+------+
|emp_id|       name|dept_id|salary|
+------+-----------+-------+------+
|     1| Employee-1|      2| 50001|
|     2| Employee-2|      3| 50002|
|     3| Employee-3|      4| 50003|
|     4| Employee-4|      5| 50004|
|     5| Employee-5|      6| 50005|
|     6| Employee-6|      7| 50006|
|     7| Employee-7|      8| 50007|
|     8| Employee-8|      9| 50008|
|     9| Employee-9|     10| 50009|
|    10|Employee-10|     11| 50010|
|    11|Employee-11|     12| 50011|
|    12|Employee-12|     13| 50012|
|    13|Employee-13|     14| 50013|
|    14|Employee-14|     15| 50014|
|    15|Employee-15|     16| 50015|
|    16|Employee-16|     17| 50016|
|    17|Employee-17|     18| 50017|
|    18|Employee-18|     19| 50018|
|    19|Employee-19|     20| 50019|
|    20|Employee-20|     21| 50020|
+------+-----------+-------+------+
only showing top 20 rows



In [0]:
print(f"부서 테이블 크기: {dept_df.count()} 행")
print(f"직원 테이블 크기: {emp_df.count()} 행")
print("="*50)

부서 테이블 크기: 100 행
직원 테이블 크기: 10000000 행


In [0]:
# 캐시하여 비교를 공정하게 만듦
dept_df.cache().count()
emp_df.cache().count()

Out[64]: 10000000

## 일반 조인

In [0]:
def perform_regular_join():
    start_time = time.time()

    regular_join = emp_df.join(
        dept_df,
        emp_df.dept_id == dept_df.dept_id
    )
        
    # 실행 트리거
    regular_count = regular_join.count()
    regular_time = time.time() - start_time
        
    print("\n일반 조인 실행:")
    print(f"처리 시간: {regular_time:.2f}초")
    print(f"결과 행 수: {regular_count}")
    print("\n실행 계획:")
    regular_join.explain()

In [0]:
perform_regular_join()


일반 조인 실행:
처리 시간: 3.43초
결과 행 수: 10000000

실행 계획:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [dept_id#14328L], [dept_id#14307L], Inner, BuildRight, false, true
   :- Filter isnotnull(dept_id#14328L)
   :  +- InMemoryTableScan [emp_id#14326L, name#14327, dept_id#14328L, salary#14329L], [isnotnull(dept_id#14328L)], false
   :        +- InMemoryRelation [emp_id#14326L, name#14327, dept_id#14328L, salary#14329L], StorageLevel(disk, memory, deserialized, 1 replicas)
   :              +- *(1) Scan ExistingRDD[emp_id#14326L,name#14327,dept_id#14328L,salary#14329L]
   +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=15455]
      +- Filter isnotnull(dept_id#14307L)
         +- InMemoryTableScan [dept_id#14307L, dept_name#14308, location#14309], [isnotnull(dept_id#14307L)], false
               +- InMemoryRelation [dept_id#14307L, dept_name#14308, location#14309], StorageLevel(disk, memory, deserialized, 1 replicas)
                     +- *(1) Scan Exis

In [0]:
# 브로드캐스트 자동 최적화 비활성화
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)


In [0]:
perform_regular_join()


일반 조인 실행:
처리 시간: 6.71초
결과 행 수: 10000000

실행 계획:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [dept_id#14328L], [dept_id#14307L], Inner
   :- Sort [dept_id#14328L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(dept_id#14328L, 200), ENSURE_REQUIREMENTS, [plan_id=15847]
   :     +- Filter isnotnull(dept_id#14328L)
   :        +- InMemoryTableScan [emp_id#14326L, name#14327, dept_id#14328L, salary#14329L], [isnotnull(dept_id#14328L)], false
   :              +- InMemoryRelation [emp_id#14326L, name#14327, dept_id#14328L, salary#14329L], StorageLevel(disk, memory, deserialized, 1 replicas)
   :                    +- *(1) Scan ExistingRDD[emp_id#14326L,name#14327,dept_id#14328L,salary#14329L]
   +- Sort [dept_id#14307L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(dept_id#14307L, 200), ENSURE_REQUIREMENTS, [plan_id=15848]
         +- Filter isnotnull(dept_id#14307L)
            +- InMemoryTableScan [dept_id#14307L, dept_name#14308, 

## 브로드캐스트 조인

In [0]:
start_time = time.time()
    
broadcast_join = emp_df.join(
    broadcast(dept_df),
    emp_df.dept_id == dept_df.dept_id
)
    
# 실행 트리거
broadcast_count = broadcast_join.count()
broadcast_time = time.time() - start_time
    
print("\n브로드캐스트 조인 실행:")
print(f"처리 시간: {broadcast_time:.2f}초")
print(f"결과 행 수: {broadcast_count}")
print("\n실행 계획:")
broadcast_join.explain()


브로드캐스트 조인 실행:
처리 시간: 2.72초
결과 행 수: 10000000

실행 계획:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [dept_id#14328L], [dept_id#14307L], Inner, BuildRight, false, true
   :- Filter isnotnull(dept_id#14328L)
   :  +- InMemoryTableScan [emp_id#14326L, name#14327, dept_id#14328L, salary#14329L], [isnotnull(dept_id#14328L)], false
   :        +- InMemoryRelation [emp_id#14326L, name#14327, dept_id#14328L, salary#14329L], StorageLevel(disk, memory, deserialized, 1 replicas)
   :              +- *(1) Scan ExistingRDD[emp_id#14326L,name#14327,dept_id#14328L,salary#14329L]
   +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=16165]
      +- Filter isnotnull(dept_id#14307L)
         +- InMemoryTableScan [dept_id#14307L, dept_name#14308, location#14309], [isnotnull(dept_id#14307L)], false
               +- InMemoryRelation [dept_id#14307L, dept_name#14308, location#14309], StorageLevel(disk, memory, deserialized, 1 replicas)
                     +- *(1) Scan 

In [0]:
# 결과 데이터 샘플 확인
print("\n결과 데이터 샘플:")
broadcast_join.select("emp_id", "name", "dept_name", "location", "salary").show(5)
    
# 집계 쿼리로 조인 결과 활용 예시
print("\n부서별 평균 급여:")
broadcast_join.groupBy("dept_name").agg({"salary": "avg"}).orderBy("dept_name").show(5)


결과 데이터 샘플:
+------+----------+------------+----------+------+
|emp_id|      name|   dept_name|  location|salary|
+------+----------+------------+----------+------+
|     1|Employee-1|Department-2|Location-2| 50001|
|     2|Employee-2|Department-3|Location-3| 50002|
|     3|Employee-3|Department-4|Location-4| 50003|
|     4|Employee-4|Department-5|Location-0| 50004|
|     5|Employee-5|Department-6|Location-1| 50005|
+------+----------+------------+----------+------+
only showing top 5 rows


부서별 평균 급여:
+--------------+-----------+
|     dept_name|avg(salary)|
+--------------+-----------+
|  Department-1|    74950.0|
| Department-10|    74959.0|
|Department-100|    75049.0|
| Department-11|    74960.0|
| Department-12|    74961.0|
+--------------+-----------+
only showing top 5 rows



In [0]:
dept_df.createOrReplaceTempView("dept")
emp_df.createOrReplaceTempView("emp")

In [0]:
df = spark.sql("""
SELECT e.emp_id,
       e.name,
       d.dept_name
FROM   emp e
JOIN   dept d
ON     e.dept_id = d.dept_id
""")
df.show()

df.explain(True)

+------+------------+-------------+
|emp_id|        name|    dept_name|
+------+------------+-------------+
|    25| Employee-25|Department-26|
|    28| Employee-28|Department-29|
|   125|Employee-125|Department-26|
|   128|Employee-128|Department-29|
|   225|Employee-225|Department-26|
|   228|Employee-228|Department-29|
|   325|Employee-325|Department-26|
|   328|Employee-328|Department-29|
|   425|Employee-425|Department-26|
|   428|Employee-428|Department-29|
|   525|Employee-525|Department-26|
|   528|Employee-528|Department-29|
|   625|Employee-625|Department-26|
|   628|Employee-628|Department-29|
|   725|Employee-725|Department-26|
|   728|Employee-728|Department-29|
|   825|Employee-825|Department-26|
|   828|Employee-828|Department-29|
|   925|Employee-925|Department-26|
|   928|Employee-928|Department-29|
+------+------------+-------------+
only showing top 20 rows

== Parsed Logical Plan ==
'Project ['e.emp_id, 'e.name, 'd.dept_name]
+- 'Join Inner, ('e.dept_id = 'd.dept_id

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10485760b")

df = spark.sql("""
    SELECT /*+ BROADCAST(dept) */
       e.emp_id,
       e.name,
       d.dept_name
    FROM   emp e
    JOIN   dept d
    ON     e.dept_id = d.dept_id
""")
df.show()

df.explain(True)

+------+-----------+-------------+
|emp_id|       name|    dept_name|
+------+-----------+-------------+
|     1| Employee-1| Department-2|
|     2| Employee-2| Department-3|
|     3| Employee-3| Department-4|
|     4| Employee-4| Department-5|
|     5| Employee-5| Department-6|
|     6| Employee-6| Department-7|
|     7| Employee-7| Department-8|
|     8| Employee-8| Department-9|
|     9| Employee-9|Department-10|
|    10|Employee-10|Department-11|
|    11|Employee-11|Department-12|
|    12|Employee-12|Department-13|
|    13|Employee-13|Department-14|
|    14|Employee-14|Department-15|
|    15|Employee-15|Department-16|
|    16|Employee-16|Department-17|
|    17|Employee-17|Department-18|
|    18|Employee-18|Department-19|
|    19|Employee-19|Department-20|
|    20|Employee-20|Department-21|
+------+-----------+-------------+
only showing top 20 rows

== Parsed Logical Plan ==
'UnresolvedHint BROADCAST, ['dept]
+- 'Project ['e.emp_id, 'e.name, 'd.dept_name]
   +- 'Join Inner, ('e.dep